# Initialization

In [1]:
from logging import ERROR
import os
from pathlib import Path
import signal
from subprocess import CalledProcessError
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces
from pyboolnet import log
log.setLevel(ERROR)

# mpbn init
import mpbn

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

INFO Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO NumExpr defaulting to 8 threads.


<Handlers.SIG_DFL: 0>

In [2]:
# compute trapspaces using PyBoolNet
def bench_pyboolnet(models, timeout=180, number=1, max_output=1000):
    for model in models:
        print(model)        
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                primes = bnet2primes(model + ".bnet")
                tspaces = compute_trap_spaces(primes, "min", max_output=max_output)
                nb_tspaces = len(tspaces)
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except (TimeOutException, CalledProcessError, Exception):
                total_time += timeout
            except MemoryError:
                print ("MemoryError")
                total_time += timeout
            signal.alarm(0)
        
        os.system("killall -9 BNetToPrime_linux64")
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [3]:
# compute trapspaces using mpbn
def bench_mpbn(models, timeout=180, number=1, max_output=1000):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                mbn = mpbn.load(model + ".bnet")
                tspaces = list(mbn.attractors(limit=max_output))
                nb_tspaces = len(tspaces)
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except AssertionError:
                if i == 0:
                    print("Non-monotonic")
            except TimeOutException:
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [4]:
# compute trapspaces using trappist
def bench_trappist(models, timeout=180, number=1, max_output=1000, **kwargs):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                tspaces = trappist.compute_trap_spaces(model + ".bnet", max_output=max_output, time_limit=timeout, **kwargs)
                nb_tspaces = len(list(tspaces))
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except (TimeOutException, CalledProcessError):
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

# BBM repository

In [5]:
models = [str(Path().absolute() / "TCS-data-set/BBM" / name[:-5]) for name in os.listdir("TCS-data-set/BBM") if name.endswith(".bnet")]

In [8]:
bench_pyboolnet(models)

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-183]__[var-31]__[in-4]__[ALTERATIONS-IN-BLADDER]


BNetToPrime_linux64: no process found


25 trap spaces 0.92
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-210]__[var-144]__[in-0]__[DRUG-SYNERGY-PREDICTION]


BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found


1 trap spaces 4.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-097]__[var-8]__[in-2]__[DROSOPHILA-WINGS-AP]
4 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-001]__[var-302]__[in-19]__[SIGNALING-IN-MACROPHAGE-ACTIVATION]


...

BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found


1000 trap spaces 79.18
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-019]__[var-71]__[in-15]__[IL-6-SIGNALING]
1000 trap spaces 0.09
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-161]__[var-94]__[in-2]__[DIFFERENTIATION-OF-MONOCYTES]


BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found


4 trap spaces 1.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-108]__[var-23]__[in-2]__[GEROCONVERSION]
6 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-051]__[var-69]__[in-1]__[COLITIS-ASSOCIATED-COLON-CANCER]
10 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-008]__[var-25]__[in-3]__[DEATH-RECEPTOR-SIGNALING]
27 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-073]__[var-31]__[in-2]__[LYMPHOID-AND-MYELOID-CELL-SPECIFICATION]
21 trap spaces 0.92
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-020]__[var-39]__[in-2]__[APOPTOSIS-NETWORK]
8 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-170]__[var-4]__[in-3]__[DROSOPHILA-GAP-B]
9 trap spaces 0.05
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-067]__[var-9]__[in-4]__[REGULATION-OF

BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found
BNetToPrime_linux64: no process found


In [28]:
bench_mpbn(models)

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-183]__[var-31]__[in-4]__[ALTERATIONS-IN-BLADDER]
25 trap spaces 0.11
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-210]__[var-144]__[in-0]__[DRUG-SYNERGY-PREDICTION]
1 trap spaces 0.05
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-097]__[var-8]__[in-2]__[DROSOPHILA-WINGS-AP]
Non-monotonic
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-001]__[var-302]__[in-19]__[SIGNALING-IN-MACROPHAGE-ACTIVATION]
1000 trap spaces 2.39
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-038]__[var-21]__[in-4]__[SKBR3-BREAST-CELL-LINE-LONG-TERM]
1000 trap spaces 0.21
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-148]__[var-83]__[in-0]__[AGS-CELL-FATE-DECISION]
1 trap spaces 0.07
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-130]__[var-43]__[in-34]__[RENIN-ANGIOTENSIN

1000 trap spaces 0.49
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-036]__[var-11]__[in-5]__[HCC1954-BREAST-CELL-LINE-SHORT-TERM]
274 trap spaces 0.04
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-199]__[var-30]__[in-0]__[HEPATOCELLULAR-CARCINOMA-COMPARTMENTALIZED]
7 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-143]__[var-71]__[in-26]__[BREAST-CANCER-INHIBITORS]
1000 trap spaces 0.49
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-119]__[var-13]__[in-6]__[JNK-PATHWAY]
64 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-158]__[var-7]__[in-0]__[LAMBDA-PHAGE-LYSOGENY]
2 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-153]__[var-17]__[in-1]__[CONTROL-OF-PROLIFERATION]
3 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-091]__[var-12]

3 trap spaces 0.07
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-053]__[var-13]__[in-2]__[PREDICTING-VARIABILITIES-IN-CARDIAC-GENE]
6 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-045]__[var-11]__[in-13]__[HH-PATHWAY-OF-DROSOPHILA]
1000 trap spaces 0.11
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-176]__[var-47]__[in-2]__[MYELOFIBROTIC-MICROENVIRONMENT]
4 trap spaces 0.15
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-160]__[var-76]__[in-16]__[IL17-DIFFERENTIAL-EXPRESSION]
1000 trap spaces 0.49
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-188]__[var-30]__[in-7]__[MAMMAL-SEX-DETERMINATION-2-CELL]
1000 trap spaces 0.23
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-011]__[var-40]__[in-4]__[GUARD-CELL-ABSCISIC-ACID-SIGNALING]
28 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TC

1000 trap spaces 0.14
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-056]__[var-66]__[in-25]__[IGVH-MUTATIONS-IN-LYMPHOCYTIC-LEUKEMIA]
1000 trap spaces 0.41
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-016]__[var-104]__[in-14]__[IL-1-SIGNALING]
1000 trap spaces 0.52
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-173]__[var-62]__[in-14]__[GLUCOSE-REPRESSION-SIGNALLING]
1000 trap spaces 0.35
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-128]__[var-56]__[in-34]__[PYRIMIDINE-DEPRIVATION]
1000 trap spaces 0.40
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-010]__[var-13]__[in-2]__[CARDIAC-DEVELOPMENT]
6 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-115]__[var-17]__[in-18]__[E-PROTEIN]
1000 trap spaces 0.16
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-122]__[var-74]__[in-94]__

In [6]:
bench_trappist(models)

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-183]__[var-31]__[in-4]__[ALTERATIONS-IN-BLADDER]
25 trap spaces 0.07
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-210]__[var-144]__[in-0]__[DRUG-SYNERGY-PREDICTION]
1 trap spaces 0.06
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-097]__[var-8]__[in-2]__[DROSOPHILA-WINGS-AP]
4 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-001]__[var-302]__[in-19]__[SIGNALING-IN-MACROPHAGE-ACTIVATION]
1000 trap spaces 0.27
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-038]__[var-21]__[in-4]__[SKBR3-BREAST-CELL-LINE-LONG-TERM]
1000 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-148]__[var-83]__[in-0]__[AGS-CELL-FATE-DECISION]
1 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-130]__[var-43]__[in-34]__[RENIN-ANGIOTENSI

1000 trap spaces 0.10
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-036]__[var-11]__[in-5]__[HCC1954-BREAST-CELL-LINE-SHORT-TERM]
274 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-199]__[var-30]__[in-0]__[HEPATOCELLULAR-CARCINOMA-COMPARTMENTALIZED]
7 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-143]__[var-71]__[in-26]__[BREAST-CANCER-INHIBITORS]
1000 trap spaces 0.09
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-119]__[var-13]__[in-6]__[JNK-PATHWAY]
64 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-158]__[var-7]__[in-0]__[LAMBDA-PHAGE-LYSOGENY]
2 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-153]__[var-17]__[in-1]__[CONTROL-OF-PROLIFERATION]
3 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-091]__[var-12]

1000 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-176]__[var-47]__[in-2]__[MYELOFIBROTIC-MICROENVIRONMENT]
4 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-160]__[var-76]__[in-16]__[IL17-DIFFERENTIAL-EXPRESSION]
1000 trap spaces 0.08
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-188]__[var-30]__[in-7]__[MAMMAL-SEX-DETERMINATION-2-CELL]
1000 trap spaces 0.05
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-011]__[var-40]__[in-4]__[GUARD-CELL-ABSCISIC-ACID-SIGNALING]
28 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-133]__[var-8]__[in-2]__[ROOT-STEM-CELL-2010]
10 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-017]__[var-41]__[in-9]__[DIFFERENTIATION-OF-T-LYMPHOCYTES]
1000 trap spaces 0.05
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/

1000 trap spaces 0.10
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-173]__[var-62]__[in-14]__[GLUCOSE-REPRESSION-SIGNALLING]
1000 trap spaces 0.06
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-128]__[var-56]__[in-34]__[PYRIMIDINE-DEPRIVATION]
1000 trap spaces 0.08
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-010]__[var-13]__[in-2]__[CARDIAC-DEVELOPMENT]
6 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-115]__[var-17]__[in-18]__[E-PROTEIN]
1000 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-122]__[var-74]__[in-94]__[NSP14]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-057]__[var-15]__[in-0]__[FANCONI-ANEMIA-AND-CHECKPOINT-RECOVERY]
1 trap spaces 0.06
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-107]__[var-32]__[in-1]__[DNA-DAMAGE-INDUCED-AUTOP

In [7]:
bench_trappist(models, method='sat')

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-183]__[var-31]__[in-4]__[ALTERATIONS-IN-BLADDER]
25 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-210]__[var-144]__[in-0]__[DRUG-SYNERGY-PREDICTION]
1 trap spaces 0.04
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-097]__[var-8]__[in-2]__[DROSOPHILA-WINGS-AP]
4 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-001]__[var-302]__[in-19]__[SIGNALING-IN-MACROPHAGE-ACTIVATION]
1000 trap spaces 2.31
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-038]__[var-21]__[in-4]__[SKBR3-BREAST-CELL-LINE-LONG-TERM]
1000 trap spaces 0.05
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-148]__[var-83]__[in-0]__[AGS-CELL-FATE-DECISION]
1 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-130]__[var-43]__[in-34]__[RENIN-ANGIOTENSI

1000 trap spaces 0.39
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-036]__[var-11]__[in-5]__[HCC1954-BREAST-CELL-LINE-SHORT-TERM]
274 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-199]__[var-30]__[in-0]__[HEPATOCELLULAR-CARCINOMA-COMPARTMENTALIZED]
7 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-143]__[var-71]__[in-26]__[BREAST-CANCER-INHIBITORS]
1000 trap spaces 0.39
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-119]__[var-13]__[in-6]__[JNK-PATHWAY]
64 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-158]__[var-7]__[in-0]__[LAMBDA-PHAGE-LYSOGENY]
2 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-153]__[var-17]__[in-1]__[CONTROL-OF-PROLIFERATION]
3 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-091]__[var-12]

1000 trap spaces 0.06
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-176]__[var-47]__[in-2]__[MYELOFIBROTIC-MICROENVIRONMENT]
4 trap spaces 0.03
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-160]__[var-76]__[in-16]__[IL17-DIFFERENTIAL-EXPRESSION]
1000 trap spaces 0.33
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-188]__[var-30]__[in-7]__[MAMMAL-SEX-DETERMINATION-2-CELL]
1000 trap spaces 0.11
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-011]__[var-40]__[in-4]__[GUARD-CELL-ABSCISIC-ACID-SIGNALING]
28 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-133]__[var-8]__[in-2]__[ROOT-STEM-CELL-2010]
10 trap spaces 0.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-017]__[var-41]__[in-9]__[DIFFERENTIATION-OF-T-LYMPHOCYTES]
1000 trap spaces 0.16
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/

1000 trap spaces 0.39
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-173]__[var-62]__[in-14]__[GLUCOSE-REPRESSION-SIGNALLING]
1000 trap spaces 0.23
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-128]__[var-56]__[in-34]__[PYRIMIDINE-DEPRIVATION]
1000 trap spaces 0.25
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-010]__[var-13]__[in-2]__[CARDIAC-DEVELOPMENT]
6 trap spaces 0.01
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-115]__[var-17]__[in-18]__[E-PROTEIN]
1000 trap spaces 0.08
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-122]__[var-74]__[in-94]__[NSP14]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-057]__[var-15]__[in-0]__[FANCONI-ANEMIA-AND-CHECKPOINT-RECOVERY]
1 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-107]__[var-32]__[in-1]__[DNA-DAMAGE-INDUCED-AUTOP

In [6]:
bench_trappist(models, method='cp')

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-183]__[var-31]__[in-4]__[ALTERATIONS-IN-BLADDER]
25 trap spaces 5.52
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-210]__[var-144]__[in-0]__[DRUG-SYNERGY-PREDICTION]
1 trap spaces 0.52
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-097]__[var-8]__[in-2]__[DROSOPHILA-WINGS-AP]
4 trap spaces 0.95
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-001]__[var-302]__[in-19]__[SIGNALING-IN-MACROPHAGE-ACTIVATION]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-038]__[var-21]__[in-4]__[SKBR3-BREAST-CELL-LINE-LONG-TERM]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-148]__[var-83]__[in-0]__[AGS-CELL-FATE-DECISION]
1 trap spaces 0.46
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-130]__[var-43]__[in-34]__[RENIN-ANGIOTENSIN]
Timeout
DNF
===
/

8 trap spaces 2.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-059]__[var-62]__[in-5]__[BORTEZOMIB-RESPONSES-IN-MYELOMA-CELLS]
83 trap spaces 21.54
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-080]__[var-95]__[in-15]__[TCR-SIGNALING-2018]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-036]__[var-11]__[in-5]__[HCC1954-BREAST-CELL-LINE-SHORT-TERM]
274 trap spaces 65.66
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-199]__[var-30]__[in-0]__[HEPATOCELLULAR-CARCINOMA-COMPARTMENTALIZED]
7 trap spaces 1.96
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-143]__[var-71]__[in-26]__[BREAST-CANCER-INHIBITORS]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-119]__[var-13]__[in-6]__[JNK-PATHWAY]
64 trap spaces 15.41
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-158]__[var-7]__[in

26 trap spaces 6.59
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-174]__[var-19]__[in-0]__[HEPATOCELLULAR-CARCINOMA-REDUCED]
13 trap spaces 3.18
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-047]__[var-14]__[in-9]__[FGF-PATHWAY-OF-DROSOPHILA]
512 trap spaces 130.05
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-037]__[var-11]__[in-5]__[SKBR3-BREAST-CELL-LINE-SHORT-TERM]
702 trap spaces 162.12
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-054]__[var-61]__[in-1]__[PC12-CELL-DIFFERENTIATION]
3 trap spaces 0.84
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-181]__[var-12]__[in-1]__[MULTILEVEL-CELL-CYCLE]
3 trap spaces 0.92
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-053]__[var-13]__[in-2]__[PREDICTING-VARIABILITIES-IN-CARDIAC-GENE]
6 trap spaces 1.50
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BB

27 trap spaces 6.30
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-111]__[var-18]__[in-15]__[APOPTOSIS]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-012]__[var-94]__[in-7]__[T-CELL-RECEPTOR-SIGNALING]
128 trap spaces 34.33
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-138]__[var-17]__[in-14]__[APOPTOSIS-UPDATED]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-056]__[var-66]__[in-25]__[IGVH-MUTATIONS-IN-LYMPHOCYTIC-LEUKEMIA]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-016]__[var-104]__[in-14]__[IL-1-SIGNALING]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-173]__[var-62]__[in-14]__[GLUCOSE-REPRESSION-SIGNALLING]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-128]__[var-56]__[in-34]__[PYRIMIDINE-DEPRIVATION]
Timeout
DNF
===
/home/g

In [6]:
bench_trappist(models, method='ilp')

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-183]__[var-31]__[in-4]__[ALTERATIONS-IN-BLADDER]
25 trap spaces 7.40
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-210]__[var-144]__[in-0]__[DRUG-SYNERGY-PREDICTION]
1 trap spaces 0.48
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-097]__[var-8]__[in-2]__[DROSOPHILA-WINGS-AP]
4 trap spaces 1.00
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-001]__[var-302]__[in-19]__[SIGNALING-IN-MACROPHAGE-ACTIVATION]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-038]__[var-21]__[in-4]__[SKBR3-BREAST-CELL-LINE-LONG-TERM]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-148]__[var-83]__[in-0]__[AGS-CELL-FATE-DECISION]
1 trap spaces 0.34
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-130]__[var-43]__[in-34]__[RENIN-ANGIOTENSIN]
Timeout
DNF
===
/

8 trap spaces 1.47
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-059]__[var-62]__[in-5]__[BORTEZOMIB-RESPONSES-IN-MYELOMA-CELLS]
83 trap spaces 162.26
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-080]__[var-95]__[in-15]__[TCR-SIGNALING-2018]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-036]__[var-11]__[in-5]__[HCC1954-BREAST-CELL-LINE-SHORT-TERM]
274 trap spaces 68.78
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-199]__[var-30]__[in-0]__[HEPATOCELLULAR-CARCINOMA-COMPARTMENTALIZED]
7 trap spaces 1.59
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-143]__[var-71]__[in-26]__[BREAST-CANCER-INHIBITORS]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-119]__[var-13]__[in-6]__[JNK-PATHWAY]
64 trap spaces 11.39
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-158]__[var-7]__[i

26 trap spaces 5.40
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-174]__[var-19]__[in-0]__[HEPATOCELLULAR-CARCINOMA-REDUCED]
13 trap spaces 2.67
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-047]__[var-14]__[in-9]__[FGF-PATHWAY-OF-DROSOPHILA]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-037]__[var-11]__[in-5]__[SKBR3-BREAST-CELL-LINE-SHORT-TERM]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-054]__[var-61]__[in-1]__[PC12-CELL-DIFFERENTIATION]
3 trap spaces 1.14
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-181]__[var-12]__[in-1]__[MULTILEVEL-CELL-CYCLE]
3 trap spaces 0.93
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-053]__[var-13]__[in-2]__[PREDICTING-VARIABILITIES-IN-CARDIAC-GENE]
6 trap spaces 1.53
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-045]__[var-11]__

27 trap spaces 6.23
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-111]__[var-18]__[in-15]__[APOPTOSIS]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-012]__[var-94]__[in-7]__[T-CELL-RECEPTOR-SIGNALING]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-138]__[var-17]__[in-14]__[APOPTOSIS-UPDATED]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-056]__[var-66]__[in-25]__[IGVH-MUTATIONS-IN-LYMPHOCYTIC-LEUKEMIA]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-016]__[var-104]__[in-14]__[IL-1-SIGNALING]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-173]__[var-62]__[in-14]__[GLUCOSE-REPRESSION-SIGNALLING]
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/BBM/[id-128]__[var-56]__[in-34]__[PYRIMIDINE-DEPRIVATION]
Timeout
DNF
===
/home/giang-trinh